# Answering questions

Create a model capable of answering questions with citations using knowledge base.

## Imports

In [2]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

## Reader model

Hugging face utilisent le modèle zephyr-7b-beta qui date un peu, et ils le quantize. Pour quantizer il faut un GPU. Je suppose qu'une fois la quantization faite on peut charger le modèle sur CPU.

Pour les tests j'ai pris un petit modèle pas trop mal classé dans le leaderborad. https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard#/?params=-1%2C3 


In [8]:
READER_MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/ac/b2/acb26770a261dcc4f0e397d81ddea9e06b330d79f58c385663acdae527a7230e/dd924a11b4c220f385b51ffa522daea7c9f3d850e31b162bb5661df483c6d3ee?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1742680431&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MjY4MDQzMX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2FjL2IyL2FjYjI2NzcwYTI2MWRjYzRmMGUzOTdkODFkZGVhOWUwNmIzMzBkNzlmNThjMzg1NjYzYWNkYWU1MjdhNzIzMGUvZGQ5MjRhMTFiNGMyMjBmMzg1YjUxZmZhNTIyZGFlYTdjOWYzZDg1MGUzMWIxNjJiYjU2NjFkZjQ4M2M2ZDNlZT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=VenuNEfUpIFUwjtx1us72ifDKS6D8RCvS%7E2ye4KFkBj4jSZsCyOaSoBXDFROkvGFWNpmU9TSsHS4vuZJhQiaowPxjRsaMveBhrcUDv-IGv55AP0jSpVNEdfQOH0Bn314fb9fC0JDgVkZ7K3pWpfnozrSRFoTgBEbqRrckZeb-5s4nINtPEVwn430Mw1IEY0b2exlHo4u1oa-5DUYK02DYYxHWiJai4MkWmnpP9w0-gQXC6qiyZ4rUIyT6IpsZGBGf5

model-00004-of-00008.safetensors:  28%|##8       | 545M/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:  25%|##4       | 493M/1.98G [00:00<?, ?B/s]

...

model-00002-of-00008.safetensors:  30%|##9       | 577M/1.95G [00:00<?, ?B/s]

model-00001-of-00008.safetensors:  26%|##6       | 493M/1.89G [00:00<?, ?B/s]

model.safetensors:   9%|8         | 273M/3.09G [00:00<?, ?B/s]

pytorch_model-00002-of-00006.bin:   7%|6         | 336M/4.95G [00:00<?, ?B/s]

pytorch_model-00001-of-00006.bin:   7%|6         | 336M/4.92G [00:00<?, ?B/s]

In [ ]:
READER_LLM("What is 4+4? Answer:")

## Prompt

In [ ]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

## Code from notebook *1_Retrieval_capacity.ipynb* for the retrieval

In [ ]:
# Load embeddings
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

EMBEDDING_MODEL_NAME = "thenlper/gte-small"

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cpu"},  # replace 'cpu' by 'cuda' if you have Nvidia gpu
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

KNOWLEDGE_VECTOR_DATABASE = FAISS.load_local("../outputs/rag_embeddings_thenlper_gte-small", embedding_model, allow_dangerous_deserialization=True)

## Test Reader

In [ ]:
user_query = """Your Client, A Inc, is a sub-licensee under European patent application EP-1. Can the sub-licence be recorded in the European Patent Register?
 
A    No, it is not possible to record sub-licences in the European Patent Register.
 
B    Yes, any sub-licence can be recorded in the European Patent Register.
 
C    Yes, provided the licensee granting the sub-licence has recorded its licence in the European Patent Register.
"""

Correct answer:

The correct answer is C.

A sub-licence can only be recorded in the European Patent Register if it is granted by a licensee whose licence is recorded in 
the Register (Rule 24(b) EPC, implementing Article 73 EPC).

In [ ]:
print(f"\nStarting retrieval for {user_query=}...")
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=5)
print("\n========================== Retrieved Documents ==========================")
for i, doc in enumerate(retrieved_docs, start=1):
    print(f"\n========================== Document {i} ==========================")
    print(doc.page_content)
    print("\n========================== Metadata ==========================")
    print(doc.metadata)

In [ ]:
retrieved_docs_text = [doc.page_content for doc in retrieved_docs]  # We only need the text of the documents
context = "\nExtracted documents:\n"
context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)])

final_prompt = RAG_PROMPT_TEMPLATE.format(question="How to create a pipeline object?", context=context)

# Redact an answer
answer = READER_LLM(final_prompt)[0]["generated_text"]
print(answer)

## Reranking

A good option for RAG is to retrieve more documents than you want in the end, then rerank the results with a more powerful retrieval model before keeping only the top_k.

In [ ]:
from ragatouille import RAGPretrainedModel

RERANKER = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

## Assemble it all !

In [ ]:
from transformers import Pipeline
from langchain.docstore.document import Document as LangchainDocument
from typing import Optional, List, Tuple


def answer_with_rag(
    question: str,
    llm: Pipeline,
    knowledge_index: FAISS,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 5,
) -> Tuple[str, List[LangchainDocument]]:
    # Gather documents with retriever
    print("=> Retrieving documents...")
    relevant_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
    relevant_docs = [doc.page_content for doc in relevant_docs]  # Keep only the text

    # Optionally rerank results
    if reranker:
        print("=> Reranking documents...")
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Redact an answer
    print("=> Generating answer...")
    answer = llm(final_prompt)[0]["generated_text"]

    return answer, relevant_docs

In [ ]:
# Test
user_query = """Your Client, A Inc, is a sub-licensee under European patent application EP-1. Can the sub-licence be recorded in the European Patent Register?
 
A    No, it is not possible to record sub-licences in the European Patent Register.
 
B    Yes, any sub-licence can be recorded in the European Patent Register.
 
C    Yes, provided the licensee granting the sub-licence has recorded its licence in the European Patent Register.
"""

answer, relevant_docs = answer_with_rag(user_query, READER_LLM, KNOWLEDGE_VECTOR_DATABASE, reranker=RERANKER)

print("==================================Answer==================================")
print(f"{answer}")
print("==================================Source docs==================================")
for i, doc in enumerate(relevant_docs):
    print(f"Document {i}------------------------------------------------------------")
    print(doc)